In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
data = torch.ones(size=(10,1,32,32))

In [3]:
class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,6,5) # 1 + (5-1) * (1) = 5
        self.pool1 = nn.MaxPool2d(kernel_size=2,stride=2) #5 + (2 - 1) * (1 * 1) = 6
        self.conv2 = nn.Conv2d(6,16,5) # 6 + (5 - 1) * 2 = 14
        self.pool2 = nn.MaxPool2d(kernel_size=2,stride=2) #14 + (2 - 1) * (2 * 1) = 16
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
    
    def forward(self,x):
        x = F.tanh(self.conv1(x))
        x = self.pool1(x)
        x = F.tanh(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1,16*5*5)
        x = F.tanh(self.fc1(x))
        output = F.softmax(self.fc2(x),dim=1)
        output = F.softmax(x.view(-1,16*5*5),dim=1)

In [5]:
data = torch.ones(size=(10,3,227,227)) #假设图像的尺寸为227x227

In [ ]:
#每一层的感受野的计算公式：
#这一层的感受野 = 上一层的感受野 + （这一层的核尺寸-1）*连乘（从最初的一层到上一层的步长）
#输入图像的r0 = 1, s0 = 1

In [6]:
class AlexNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        #大卷积核、较大的步长、较多的通道
        self.conv1 = nn.Conv2d(3,96,kernel_size=11, stride=4) #1 + (11-1) * (1) = 11
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=2) #11 + (3-1) * (1 * 4) = 19
        
        #卷积核、步长恢复正常大小，进一步扩大通道
        self.conv2 = nn.Conv2d(96,256,kernel_size=5, padding=2) #19 + (5-1) * (1 * 4 * 2) = 51
        self.pool2 = nn.MaxPool2d(kernel_size=3,stride=2) #51 + (3-1) * (1 * 4 * 2 * 1) = 67
        
        #连续的卷积层，疯狂提取特征
        self.conv3 = nn.Conv2d(256,384,kernel_size=3,padding=1)
        self.conv4 = nn.Conv2d(384,384,kernel_size=3,padding=1)
        self.conv5 = nn.Conv2d(384,256,kernel_size=3,padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=3,stride=2)
        
        #全连接层
        self.fc1 = nn.Linear(256*6*6,4096) #这里的上层输入是图像中的全部像素
        self.fc2 = nn.Linear(4096,4096)
        self.fc3 = nn.Linear(4096,1000) #输出ImageNet的一千个类别
    
    def forward(self,x):
        
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.pool3(x)
        
        x = x.view(-1,256*6*6) #需要将数据的特征部分“拉平”才能够进入FC层
        
        x = F.relu(F.dropout(self.fc1(x),0.5)) #dropout：随机让50%的权重为0
        x = F.relu(F.dropout(self.fc2(x),0.5)) 
        output = F.softmax(self.fc3(x),dim=1)

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_receptive_field import receptive_field

In [10]:
class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,6,5) # 1 + (5-1) * (1) = 5
        self.pool1 = nn.MaxPool2d(kernel_size=2,stride=2) #5 + (2 - 1) * (1 * 1) = 6
        self.conv2 = nn.Conv2d(6,16,5) # 6 + (5 - 1) * 2 = 14
        self.pool2 = nn.MaxPool2d(kernel_size=2,stride=2) #14 + (2 - 1) * (2 * 1) = 16
        #self.fc1 = nn.Linear(16*5*5,120)
        #self.fc2 = nn.Linear(120,84)
    
    def forward(self,x):
        x = F.tanh(self.conv1(x))
        x = self.pool1(x)
        x = F.tanh(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1,16*5*5)
        #x = F.tanh(self.fc1(x))
        #output = F.softmax(self.fc2(x),dim=1)
        #output = F.softmax(x.view(-1,16*5*5),dim=1)

In [13]:
net = LeNet5().cuda()

In [15]:
receptive_field_dict = receptive_field(net,(1,32,32)) #输入的数据结构，这里的输入不包括样本数

------------------------------------------------------------------------------
        Layer (type)    map size      start       jump receptive_field 
        0               [32, 32]        0.5        1.0             1.0 
        1               [28, 28]        2.5        1.0             5.0 
        2               [14, 14]        3.0        2.0             6.0 
        3               [10, 10]        7.0        2.0            14.0 
        4                 [5, 5]        8.0        4.0            16.0 


#当你的PC上有GPU的时候，receptive_field函数会自动在gpu上运行，因此，你必须把输入这个函数的网络放到gpu上，才可以顺利运行

In [ ]:
#parameters = (Kh * Kw * Cin) * Cout + Cout 

In [2]:
import torch
from torch import nn

In [3]:
conv1 = nn.Conv2d(3,6,3) #3 * 3 * 3 * 6 + 6 = 168
conv2 = nn.Conv2d(6,4,3) #(3 * 3 * 6) * 4 + 4 = 220

In [5]:
conv1.weight.numel()

162

In [6]:
conv1.bias.numel()

6

In [7]:
conv2.weight.numel()

216

In [8]:
conv2.bias.numel()

4

In [9]:
conv3 = nn.Conv2d(4,16,5,stride=2,padding=1) #(5 * 5 * 4)*16 + 16 = 1616

In [10]:
conv3.weight.numel()

1600

In [12]:
conv3.bias.numel()

16

In [ ]:
#4个输入，8个输出，kz = 3x3

In [23]:
conv1 = nn.Conv2d(4,8,3) #288 + 8 =296
conv1_ = nn.Conv2d(4,8,3,groups=2) #144 + 8 = 152

In [24]:
conv1.weight.numel()

288

In [25]:
conv1_.weight.numel()

144

In [ ]:
#普通卷积 = (ks^2 * C_in) * C_out
#分组卷积 = 1/g * (ks^2 * C_in * C_out)，深度卷积时g = C_in
#逐点卷积/1x1卷积 = C_in * C_out
#深度可分离卷积 = ks^2 * C_in_depth + C_in_pair * C_out_pair
#比例 = 1/C_in_depth + C_out_pair/(ks^2 * C_in_depth)

In [26]:
conv1 = nn.Conv2d(4,8,3, bias=False) #(3*3 * 4 * 8) = 288

In [28]:
conv1_depthwise = nn.Conv2d(4,8,3,groups=4,bias=False) #1/4 * 288 = 72
conv1_pairwise = nn.Conv2d(8,8,1,bias=False) #64

In [29]:
1/4 + 8/(9 * 4)

0.4722222222222222

In [31]:
(conv1_depthwise.weight.numel() + conv1_pairwise.weight.numel())/conv1.weight.numel()

0.4722222222222222

In [2]:
import torch
from torch import nn

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/72.png?versionId=CAEQFRiBgICvypagxxciIDdhZWI1ODNjOGUzYTQ1YTFhOTNmMGQ3MzdlOGVlMTA2)

In [13]:
data = torch.ones(size=(10,3,229,229))

In [14]:
net = nn.Sequential(nn.Conv2d(3,6,3)
                    ,nn.ReLU(inplace=True)
                    ,nn.Conv2d(6,4,3)
                    ,nn.ReLU(inplace=True)
                    ,nn.MaxPool2d(2)
                    ,nn.Conv2d(4,16,5,stride=2,padding=1)
                    ,nn.ReLU(inplace=True)
                    ,nn.Conv2d(16,3,5,stride=3,padding=2)
                    ,nn.ReLU(inplace=True)
                    ,nn.MaxPool2d(2)
                   )

In [15]:
net(data).shape #卷积+池化操作之后得到的特征图尺寸的大小以及特征图的数量

torch.Size([10, 3, 9, 9])

In [16]:
from torch_receptive_field import receptive_field

In [17]:
rfdict = receptive_field(net,(3,229,229))

------------------------------------------------------------------------------
        Layer (type)    map size      start       jump receptive_field 
        0             [229, 229]        0.5        1.0             1.0 
        1             [227, 227]        1.5        1.0             3.0 
        2             [227, 227]        1.5        1.0             3.0 
        3             [225, 225]        2.5        1.0             5.0 
        4             [225, 225]        2.5        1.0             5.0 
        5             [112, 112]        3.0        2.0             6.0 
        6               [55, 55]        5.0        4.0            14.0 
        7               [55, 55]        5.0        4.0            14.0 
        8               [19, 19]        5.0       12.0            30.0 
        9               [19, 19]        5.0       12.0            30.0 
        10                [9, 9]       11.0       24.0            42.0 


![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/7.png?versionId=CAEQFRiBgMD2zKyfxxciIGZhMDQ0Y2UyYTA5ZjQ1NjhhMWNjNDQ1Njg3YTFiODZh)

In [25]:
class VGG16(nn.Module):
    def __init__(self):
        super().__init__()
        self.features_ = nn.Sequential(nn.Conv2d(3,64,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(64,64,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                       
                                       ,nn.Conv2d(64,128,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(128,128,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                       
                                       ,nn.Conv2d(128,256,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(256,256,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(256,256,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                       
                                       ,nn.Conv2d(256,512,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                       
                                       ,nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                      )
        self.clf_ = nn.Sequential(nn.Dropout(0.5)
                                  ,nn.Linear(512*7*7,4096),nn.ReLU(inplace=True)
                                  ,nn.Dropout(0.5)
                                  ,nn.Linear(4096,4096),nn.ReLU(inplace=True)
                                  ,nn.Linear(4096,1000),nn.Softmax(dim=1)
                                 )
    
    def forward(self,x):
        x = self.features_(x) #用特征提取的架构提取特征
        x = x.view(-1,512*7*7) #调整数据结构，拉平数据
        output = self.clf_(x)
        return output

In [26]:
vgg = VGG16()

In [28]:
from torchinfo import summary

In [29]:
summary(vgg,input_size=(10,3,224,224),device="cpu")

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [10, 512, 7, 7]           --
|    └─Conv2d: 2-1                       [10, 64, 224, 224]        1,792
|    └─ReLU: 2-2                         [10, 64, 224, 224]        --
|    └─Conv2d: 2-3                       [10, 64, 224, 224]        36,928
|    └─ReLU: 2-4                         [10, 64, 224, 224]        --
|    └─MaxPool2d: 2-5                    [10, 64, 112, 112]        --
|    └─Conv2d: 2-6                       [10, 128, 112, 112]       73,856
|    └─ReLU: 2-7                         [10, 128, 112, 112]       --
|    └─Conv2d: 2-8                       [10, 128, 112, 112]       147,584
|    └─ReLU: 2-9                         [10, 128, 112, 112]       --
|    └─MaxPool2d: 2-10                   [10, 128, 56, 56]         --
|    └─Conv2d: 2-11                      [10, 256, 56, 56]         295,168
|    └─ReLU: 2-12                        [10, 256, 56, 56]      

In [ ]:
data = torch.ones(10,7,7)

gap = nn.AvgPool2d(7)

gap(data).shape

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/58.png?versionId=CAEQFRiBgIC0zIWfxxciIGE0ZjkyMzI1ZTE3MzQ3MmQ5NDhiZWFlN2UyMmFiYTQ0)

In [30]:
import torch
from torch import nn
from torchinfo import summary

In [31]:
data = torch.ones(size=(10,3,32,32))

In [35]:
class NiN(nn.Module):
    def __init__(self):
        super().__init__()
        self.block1 = nn.Sequential(nn.Conv2d(3,192,5,padding=2),nn.ReLU(inplace=True)
                                    ,nn.Conv2d(192,160,1),nn.ReLU(inplace=True)
                                    ,nn.Conv2d(160,96,1),nn.ReLU(inplace=True)
                                    ,nn.MaxPool2d(kernel_size=3,stride=2)
                                    ,nn.Dropout(0.25))
        self.block2 = nn.Sequential(nn.Conv2d(96,192,5,padding=2),nn.ReLU(inplace=True)
                                    ,nn.Conv2d(192,192,1),nn.ReLU(inplace=True)
                                    ,nn.Conv2d(192,192,1),nn.ReLU(inplace=True)
                                    ,nn.MaxPool2d(kernel_size=3,stride=2)
                                    ,nn.Dropout(0.25))
        self.block3 = nn.Sequential(nn.Conv2d(192,192,3,padding=1),nn.ReLU(inplace=True)
                                    ,nn.Conv2d(192,192,1),nn.ReLU(inplace=True)
                                    ,nn.Conv2d(192,10,1),nn.ReLU(inplace=True)
                                    ,nn.AvgPool2d(7,stride=1)
                                    ,nn.Softmax(dim=1))
    def forward(self,x):
        output = self.block3(self.block2(self.block1(x)))
        return output

In [36]:
net= NiN()

In [37]:
net(data).shape #10个特征图，每个特征图尺寸是1x1

torch.Size([10, 10, 1, 1])

In [38]:
summary(net,(10,3,32,32),device="cpu")

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [10, 96, 15, 15]          --
|    └─Conv2d: 2-1                       [10, 192, 32, 32]         14,592
|    └─ReLU: 2-2                         [10, 192, 32, 32]         --
|    └─Conv2d: 2-3                       [10, 160, 32, 32]         30,880
|    └─ReLU: 2-4                         [10, 160, 32, 32]         --
|    └─Conv2d: 2-5                       [10, 96, 32, 32]          15,456
|    └─ReLU: 2-6                         [10, 96, 32, 32]          --
|    └─MaxPool2d: 2-7                    [10, 96, 15, 15]          --
|    └─Dropout: 2-8                      [10, 96, 15, 15]          --
├─Sequential: 1-2                        [10, 192, 7, 7]           --
|    └─Conv2d: 2-9                       [10, 192, 15, 15]         460,992
|    └─ReLU: 2-10                        [10, 192, 15, 15]         --
|    └─Conv2d: 2-11                      [10, 192, 15, 15]         3